# Defining the connection with CustomVision API

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials

ENDPOINT = "XXXXXXXXXX"

# Replace with a valid key
training_key = "XXXXXXXXX"
prediction_key = "XXXXXXXXXX"
prediction_resource_id = "XXXXXXXXXX"

publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Final_assignment")

# Generating Tags

In [ ]:
# Make two tags in the new project
adidas_tag = trainer.create_tag(project.id, "Adidas")
nike_tag = trainer.create_tag(project.id, "Nike")
puma_tag = trainer.create_tag(project.id, "Puma")

# Uploading Images

In [ ]:
import os
base_image_url = "C:/Users/altha/althaf_docs/Sem 2/Social Data Mining Techniques/Final Assignment/Final Assignment - Images"

print("Adding images...")

image_list = []
for file in os.listdir(base_image_url+"/Adidas/"):
    with open(base_image_url+"/Adidas/"+ file, "rb") as image_contents:
        print("Adidas/" + file)
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[adidas_tag.id]))


for file in os.listdir(base_image_url+"/Nike/"):
    print("Nike/" + file)
    with open(base_image_url+"/Nike/"+file, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[nike_tag.id]))


for file in os.listdir(base_image_url+"/Puma/"):
    print("Puma/" + file)
    with open(base_image_url+"/Puma/"+file, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[puma_tag.id]))
        
upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

# Training

In [ ]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

# Prediction

In [10]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry

ENDPOINT = "XXXXXXXXXX"
training_key = "XXXXXXXXXXXXXX"
prediction_key = "XXXXXXXXXXXXX"
prediction_resource_id = "XXXXXXXXXXXXXXXXXXXX"

publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

project = trainer.get_project("611bc936-654c-4a48-82e9-703ed40708a7")

with open(r"C:\Users\altha\althaf_docs\Sem 2\Social Data Mining Techniques\Final Assignment\Final Assignment - Images\Puma\test\shoes_ia56544.jpg", "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    
    print("\t" + results.predictions[0].tag_name +
              ": {0:.2f}%".format(results.predictions[0].probability * 100))


	Puma: 13.98%
